In [52]:
import pandas as pd
import re

In [53]:
# 1. 데이터 로드
phishing_df = pd.read_csv('init_phishing.csv', usecols=["Spam message"], encoding='cp949')
normal_df = pd.read_csv('generated_ads_cards_messages.csv', encoding='utf-8')

In [54]:
phishing_df

,Spam message
0,[국외발신]ifg@본인**세/***cm/**kg/C컵/No콤/질*/사진은 라인으로확...
1,[Web발신]ifg@(광고)[KT] 당신의 일상을 업그레이드할 기회!ifg@생활에 ...
2,[국외발신]ifg@久しぶりです。以前の古いIDは使いません。私の新しいIDを追加してくださ...
3,[Web발신]ifg@(광고) 입생로랑 뷰티 **** 마일리지 기프트ifg@ifg@신...
4,[Web발신]ifg@(광고)[KT] **월 *주차 WEEKLY BESTifg@ifg...
...,...
49994,[Web발신]ifg@(광고) 생활공작소 BLACK FRIDAY ~**%할인ifg@i...
49995,[Web발신]ifg@(광고)[KT] D-* SUPER FLEXDAY **/**(금)...
49996,[Web발신]ifg@(광고)[KT] **월 *주차 WEEKLY BESTifg@ifg...
49997,[Web발신]ifg@(광고)Temu: ifg@사은품 증정타임: *개 무료 사은품if...


In [55]:
normal_df

,message
0,[Web발신]특별히 현대카드 회원님을 위한 [이벤트]가 진행됩니다.현대카드 혜택을 ...
1,특별히 하나카드 회원님을 위한 [이벤트]가 진행됩니다.하나카드 혜택을 지금 바로 확...
2,"안녕하세요 고객님, KB국민카드의 가을 맞이 혜택 이벤트 안내드립니다. 적용 기간:..."
3,특별히 신한카드 회원님을 위한 [이벤트]가 진행됩니다.신한카드 혜택을 지금 바로 확...
4,"[Web발신]안녕하세요 고객님, 롯데카드의 봄 맞이 혜택 이벤트 안내드립니다. 적용..."
...,...
39995,"[Web 발신] [우리은행] 고객 감사 이벤트\n안녕하세요, 우리은행에서 진행하는 ..."
39996,"[Web 발신] [신한은행] 취업 특강\n안녕하세요, 신한은행에서 진행하는 취업 특..."
39997,"[KB국민은행] 고객 감사 이벤트\n안녕하세요, KB국민은행에서 진행하는 고객 감사..."
39998,"[신한은행] 고객 감사 이벤트\n안녕하세요, 신한은행에서 진행하는 고객 감사 이벤트..."


In [ ]:
# 2. 메시지 컬럼 통일
phishing_df['message'] = phishing_df['Spam message']
normal_df['message'] = normal_df['message']

# 3. 라벨 지정 
phishing_df['label'] = 1 
normal_df['label'] = 0

# 4. 병합
combined_df = pd.concat([phishing_df, normal_df], ignore_index=True)

# 5. 필요한 컬럼만 유지
combined_df = combined_df[['message', 'label']]

combined_df

,message,label
0,[국외발신]ifg@본인**세/***cm/**kg/C컵/No콤/질*/사진은 라인으로확...,1
1,[Web발신]ifg@(광고)[KT] 당신의 일상을 업그레이드할 기회!ifg@생활에 ...,1
2,[국외발신]ifg@久しぶりです。以前の古いIDは使いません。私の新しいIDを追加してくださ...,1
3,[Web발신]ifg@(광고) 입생로랑 뷰티 **** 마일리지 기프트ifg@ifg@신...,1
4,[Web발신]ifg@(광고)[KT] **월 *주차 WEEKLY BESTifg@ifg...,1
...,...,...
89994,"[Web 발신] [우리은행] 고객 감사 이벤트\n안녕하세요, 우리은행에서 진행하는 ...",0
89995,"[Web 발신] [신한은행] 취업 특강\n안녕하세요, 신한은행에서 진행하는 취업 특...",0
89996,"[KB국민은행] 고객 감사 이벤트\n안녕하세요, KB국민은행에서 진행하는 고객 감사...",0
89997,"[신한은행] 고객 감사 이벤트\n안녕하세요, 신한은행에서 진행하는 고객 감사 이벤트...",0


In [ ]:
#6. 데이터 전처리
from konlpy.tag import Okt      #konlpy 라이브러리를 활용해 형태소 분석 및 불용어 제거를 추가하였음.
okt = Okt()                     

def clean_text(text):
    text = re.sub(r'ifg@|\*+|[^가-힣a-zA-Z\s]', '', text)
    return text.strip()

def preprocess_user_input(user_input): # 유저 입력 전처리 함수
    cleaned = clean_text(user_input)
    tokenized = tokenize_and_filter(cleaned)
    return tokenized

# 한국어 불용어 리스트
stopwords = [
    '그', '이', '저', '것', '수', '등', '들', '및', '그리고', '또한', '에서', '하다',
    '입니다', '하지만', '그러나', '하면', '되다', '입니다', '있다', '없다',
    '입니다', '입니다만', '하지만', '어떤', '아무', '그런', '즉', '때문에', '위해'
]

# 형태소 분석 및 불용어 제거 함수
def tokenize_and_filter(text):
    tokens = okt.morphs(text, stem=True)  # 형태소 분석 + 원형 복원
    filtered = [word for word in tokens if word not in stopwords and len(word) > 1]
    return ' '.join(filtered)

import swifter
swifter.set_defaults(display_progress=True)
swifter.set_defaults(allow_dask_on_strings=False)

combined_df['message'] = (
    combined_df['message']
    .astype(str)
    .swifter.apply(lambda x: tokenize_and_filter(clean_text(x))) #Swifter를 활용한 병렬처리로 전처리 시간 향상 
)

Pandas Apply:   0%|          | 0/89999 [00:00<?, ?it/s]

In [58]:
combined_df

,message,label
0,국외 발신 인세 cmkgC No 콤질 사진 라인 으로 확인 해주다 Edex,1
1,Web 발신 광고 KT 당신 일상 업그레이드 기회 생활 필요하다 모든 만나다 보다 ...,1
2,국외 발신 IDIDlzn,1
3,Web 발신 광고 입생로랑 뷰티 마일리지 기프트 신미영 고객 고객 준비 마일리지 기...,1
4,Web 발신 광고 KT 주차 WEEKLY BEST 이번 가장 상품 여기 모이다 최대...,1
...,...,...
89994,Web 발신 우리은행 고객 감사 이벤트 안녕하다 우리은행 진행 고객 감사 이벤트 초...,0
89995,Web 발신 신한은행 취업 특강 안녕하다 신한은행 진행 취업 특강 초대 \n 장소...,0
89996,KB 국민은행 고객 감사 이벤트 안녕하다 KB 국민은행 진행 고객 감사 이벤트 초대...,0
89997,신한은행 고객 감사 이벤트 안녕하다 신한은행 진행 고객 감사 이벤트 초대 \n 장...,0


In [59]:
from sklearn.model_selection import train_test_split

X = combined_df['message']
y = combined_df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

In [60]:
X_train

76988    삼성 카드 취업 특강 안녕하다 카드 진행 취업 특강 초대 \n  장소 서울 강남구 ...
89926    KT 세미나 안내 안녕하다 KT 진행 세미나 안내 초대 \n  장소 인천 연수구 \...
51607    Web 발신 특별하다 삼성 카드 회원 이벤트 진행 삼성 카드 혜택 지금 바로 확인 ...
3801                   국외 발신 싱글 여성 무용 kgD 같다 도시 기다리다 LINgk
15531    Web 발신 광고 AK 플라자 수원 AK 플라자 수원 크리스마스 최대 쇼핑 혜택 h...
                               ...                        
39556    Web 발신 광고 KT 주차 WEEKLY BEST 이번 식품 BEST 아이템 특별 ...
65655    Web 발신 특별하다 삼성 카드 회원 이벤트 진행 삼성 카드 혜택 지금 바로 확인 ...
9616                           국제 신저 바로 당신 근처 LINExy 스릴 찾다
20374    Web 발신 광고 다비치 안경 체인 안녕하다 다비치 보청기 군산 운점 이다 저희 매...
65715    Web 발신 특별하다 우리카드 회원 이벤트 진행 우리카드 혜택 지금 바로 확인 적용...
Name: message, Length: 71999, dtype: object

In [61]:
y_train

76988    0
89926    0
51607    0
3801     1
15531    1
        ..
39556    1
65655    0
9616     1
20374    1
65715    0
Name: label, Length: 71999, dtype: int64

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=20000)  # 가장많이 출현된 상위 단어 20000개
X_train_vec = vectorizer.fit_transform(X_train)  #벡터화 문자 -> 숫자
X_test_vec = vectorizer.transform(X_test)

# 모델 학습
model = MultinomialNB()
model.fit(X_train_vec, y_train)

# 예측
y_pred = model.predict(X_test_vec)

# 평가
print(confusion_matrix(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

[[8000    0]
 [   1 9999]]
Accuracy: 0.9999444444444444
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8000
           1       1.00      1.00      1.00     10000

    accuracy                           1.00     18000
   macro avg       1.00      1.00      1.00     18000
weighted avg       1.00      1.00      1.00     18000



In [76]:
def predict_message(message):
    # 입력 메시지를 벡터화
    message_vec = vectorizer.transform([message])
    
    # 예측
    #prediction = model.predict(message_vec)
    
    # 확률 예측
    prob = model.predict_proba(message_vec)[0]  # [0]으로 첫 번째 샘플의 결과 추출
    #prediction = model.predict(message_vec)[0]  # 예측 라벨 (0 또는 1)
    
    # 결과 해석
    #result = "스팸 메시지입니다." if prediction == 1 else "정상 메시지입니다."
    spam_prob = prob[1]  # 클래스 1 (스팸)에 대한 확률
    
    return f"해당 문자가 스팸일 확률은 {spam_prob:.2%} 입니다!!"

# 3. 사용자로부터 입력 받기
user_input = input("문자를 입력하세요: ")
processed_input = preprocess_user_input(user_input)

# 예측 결과 출력
print(user_input)
print("전처리 결과:", processed_input)
print(predict_message(user_input))

제목없음 [Web발신] [진로취업과 면접합격전략 '내일부터 출근' (취업 및 면접 특강, 모의면접)안내]  결국 채용을 결정하는 건 '면접'! 유튜브 채널 '내일부터 출근'의  삼성전자 인사팀 출신 20년차 인사전문가 '제리멘토'가 알려주는 취업,  면접 꿀팁이 궁금한 학생들의 많은 참여 바랍니다.  1. 취업특강: 5/15(목) 15:00-17:00 미래창직관 대강당 2. 면접특강: 5/22(목)  15:00-17:00 미래창직관 대강당  ▶특강신청 바로가기: https://knucube.knu.ac.kr/site/lecture00112305  3. 모의면접: 5/26(월) ~ 5/29(목) 신청화면 참조 ▶모의면접 신청 바로가기: https://knucube.knu.ac.kr/site/lecture00112306 (큐브-비교과프로그램-'내일부터 출근' 검색) 4. 문의: 진로취업과 053-950-2855
전처리 결과: 제목 Web 발신 진로 취업 면접 합격 전략 내일 부터 출근 취업 면접 특강 모의 면접 안내 결국 채용 결정 면접 유튜브 채널 내일 부터 출근 삼성 전자 인사 출신 20년 인사 전문가 제리 멘토 알다 취업 면접 꿀팁 궁금하다 학생 많다 참여 바라다 취업 특강 515 15001700 미래 직관 대강 면접 특강 522 15001700 미래 직관 대강 특강 신청 바로가기 httpsknucubeknuackrsitelecture 00112305 모의 면접 526월 529 신청 화면 참조 모의 면접 신청 바로가기 httpsknucubeknuackrsitelecture 00112306 큐브 비교 프로그램 내일 부터 출근 검색 문의 진로 취업 0539502855
해당 문자가 스팸일 확률은 21.36% 입니다!!
